In [ ]:
# imports

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
from pathlib import Path
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

from config import DATALAKE

In [ ]:
# configurações fixas - parametros, diretórios, final de rota de alguns endereços

In [ ]:
raw = Path(DATALAKE) / 'raw_data_covid19' \
    / datetime.now().strftime('%Y-%m-%d') \
    / 'data-sindrome_respiratoria_aguda_grave_incluindo_covid/'

raw.mkdir(parents=True, exist_ok=True)


url = 'https://opendatasus.saude.gov.br/dataset/'

endpoints = [
    'srag-2009-2012',
    'srag-2013-2018',
    'srag-2019',
    'srag-2020',
    'srag-2021-a-2023'
]

In [ ]:
# função para usar com pull de thread

In [ ]:
def download_srag(endpoint):
    r = requests.get(f'{url}/{endpoint}')

    soup = BeautifulSoup(r.text, 'html.parser')

    tag_a = soup.findAll('a', class_='heading')

    links = []
    for tag in tag_a:
        string = tag.text.lstrip('\n').rstrip('\n').lstrip(' ').rstrip(' ')[0:4]
        if string == 'SRAG':
            links.append('/'.join(tag['href'].split('/')[2:]))


    for link in links:
        r = requests.get(f'{url}/{link}')

        soup = BeautifulSoup(r.text, 'html.parser')

        tag_a = soup.findAll('a', class_='resource-url-analytics')

        for tag in tag_a:
            if tag['href'].endswith('.csv') and tag['href'] == tag.text:
                file_csv = requests.get(tag['href'], stream=True)
                name = ''.join(tag.text.split('/')[-1:])

                with open(raw / name, 'wb') as f:
                    print(f'baixando: {name}')
                    for content in file_csv.iter_content(chunk_size=1024):
                        f.write(content)

In [ ]:
# executa a função de download e puxa os arquivos mais rapidamente
# o parametro max_workers é o número de tarefas que vão ser executadas ao mesmo tempo

In [ ]:
with ThreadPoolExecutor(max_workers=4) as executor:
    for endpoint in endpoints:
        executor.submit(download_srag, endpoint)
print(f'Arquivos baixados em {raw}')